# Loading pdf document

In [3]:
from langchain_community.document_loaders import PyPDFLoader

In [4]:
pdf_loader = PyPDFLoader(file_path="./Data/Mahabharata.pdf")

# print(pdf_loader)

docs = pdf_loader.load()
docs[0].page_content

" \nMAHABHARATA retold by C. Rajagopalachari \n(Edited by Jay Mazo, International Gita Society) \n \nContents \n \n \n1. Ganapati, the Scribe \n2. Devavrata \n3. Bhishma's Vow \n4. Amba And Bhishma \n5. Devayani And Kacha \n6. The Marriage Of Devayani \n7. Yayati \n8. Vidura \n9. Kunti Devi \n10. Death Of Pandu \n11. Bhima \n12. Karna \n13. Drona \n14. The Wax Palace \n15. The Escape Of The Pandavas \n16. The Slaying Of Bakasura \n17. Draupadi's Swayamvaram \n18. Indraprastha \n19. The Saranga Birds \n20. Jarasandha \n21. The Slaying Of Jarasandha \n22. The First Honor \n23. Sakuni Comes In \n24. The Invitation \n25. The Wager \n26. Draupadi's Grief \n27. Dhritarashtra's Anxiety \n28. Krishna's Vow \n29. Pasupata \n30. Affliction Is Nothing New \n31. Agastya \n32. Rishyasringa \n33. Fruitless Penance \n34. Yavakrida's End \n35. Mere Learning Is Not Enough \n36. Ashtavakra \n37. Bhima And Hanuman \n38. I am No Crane \n \n39. The Wicked Are Never Satisfied \n40. Duryodhana Disgraced \n41

# creating knowledgebase

In [2]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings

In [5]:
# Split docs to chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = text_spliter.split_documents(docs)
chunks

[Document(metadata={'source': './Data/Mahabharata.pdf', 'page': 0}, page_content="MAHABHARATA retold by C. Rajagopalachari \n(Edited by Jay Mazo, International Gita Society) \n \nContents \n \n \n1. Ganapati, the Scribe \n2. Devavrata \n3. Bhishma's Vow \n4. Amba And Bhishma \n5. Devayani And Kacha \n6. The Marriage Of Devayani \n7. Yayati \n8. Vidura \n9. Kunti Devi \n10. Death Of Pandu \n11. Bhima \n12. Karna \n13. Drona \n14. The Wax Palace \n15. The Escape Of The Pandavas \n16. The Slaying Of Bakasura \n17. Draupadi's Swayamvaram \n18. Indraprastha \n19. The Saranga Birds \n20. Jarasandha \n21. The Slaying Of Jarasandha \n22. The First Honor \n23. Sakuni Comes In \n24. The Invitation \n25. The Wager \n26. Draupadi's Grief \n27. Dhritarashtra's Anxiety \n28. Krishna's Vow \n29. Pasupata \n30. Affliction Is Nothing New \n31. Agastya \n32. Rishyasringa \n33. Fruitless Penance \n34. Yavakrida's End \n35. Mere Learning Is Not Enough \n36. Ashtavakra \n37. Bhima And Hanuman \n38. I am No

In [7]:
# Create embeddings
embeddings = OllamaEmbeddings(model="gemma:2b")

# creating vector and store to cromaDB
vectorstore = Chroma.from_documents(documents=chunks,embedding=embeddings, persist_directory="./chroma_db")

In [15]:
retriver = vectorstore.as_retriever()

In [16]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [13]:
from langchain_ollama.llms import OllamaLLM
model = OllamaLLM(model="gemma:2b")

In [17]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

results = rag_chain.invoke({"input": "who was arjuna"})

results

{'input': 'who was arjuna',
 'context': [Document(metadata={'page': 187, 'source': './Data/Mahabharata.pdf'}, page_content='in battle. When s ouls have left their \nbodies, there is nothing like relationship, \nnothing like brother or son or relative. \nYour sons have really no connection with \nyou. Relationship ends with death, being \nonly a bodily connection and a mere \nminor incident in the  soul\'s ete rnal life. \nFrom the nowhere do lives come, and, \nwith death,  they again disappear into \nnowhere. Why should we weep for them? \nThose who die in battle after a heroic fight \ngo as guests to receive Indra\'s hospitality. \nGrieving for what is past, you cannot gain  \nanything in the nature of dharma, pleasure \nor wealth." Thus, and in many more ways, \ndid the wise and good Vidura try to \nassuage the king\'s grief.  \nVyasa also approached Dhritarashtra \ntenderly and said: "Dear son, there is \nnothing that you do not know and which \nyou have to learn from me. You know \